In [1]:
import nibabel as nib
import numpy as np
from stl import mesh
from skimage import measure
import matplotlib as mpl

In [2]:
# Path to the nifti file (.nii, .nii.gz)
file_path = 'data/VerSe 2019/train/verse004.nii.gz'

# Extract the numpy array
nifti_file = nib.load(file_path)

np_array = nifti_file.get_fdata()
# print(np_array)
print(np_array.shape)

(161, 338, 61)


In [3]:
label_path = 'data/VerSe 2019/train/verse004_seg.nii.gz'
label_file = nib.load(label_path)

label = label_file.get_fdata()
print(label.shape)

(161, 338, 61)


In [4]:
import collections

print(collections.Counter(label.flatten()))

labeledData = np_array[label != 0]

mask = label != 0
print(mask.shape, mask.shape[0] * mask.shape[1] * mask.shape[2])

Counter({0.0: 3116756, 23.0: 29982, 24.0: 29117, 22.0: 29006, 21.0: 26865, 19.0: 20660, 20.0: 19136, 18.0: 18134, 17.0: 15732, 16.0: 14110})
(161, 338, 61) 3319498


In [5]:
from matplotlib import pyplot as plt
import tqdm
import cv2

def CTToPoint(np_array, mask):
    max_v = max(np_array.flatten())
    min_v = min(np_array.flatten())
    print(max_v, min_v)
    colors = (np_array - min_v) / (max_v - min_v)
    colors = np.expand_dims(colors, axis=-1)
    colors = np.repeat(colors, 3, axis=-1)
    # data = cv2.cvtColor(data, cv2.COLOR_GRAY2RGB)
    print(colors.shape)

    xyz = []
    colors_mask = []
    labels = []
    cmap_norm = mpl.colors.Normalize(vmin=min_v, vmax=max_v)

    for i in tqdm.tqdm(range(len(np_array))):
        for j in range(len(np_array[i])):
            for k in range(len(np_array[i][j])):
                if mask[i][j][k] == 0:
                    continue
                x, y, z = i, j, k
                xyz.append([x, y, z])
                colors_mask.append(colors[i][j][k])
                labels.append(label[i][j][k])

                # color = plt.get_cmap('brg')(cmap_norm(np_array[i][j][k]))[:3]
                # colors.append(color)


    xyz = np.array(xyz)
    colors = np.array(colors_mask)
    labels = np.expand_dims(np.array(labels), axis=-1)
    print(xyz.shape, colors.shape, labels.shape)
    np.save('data/004.npy', np.concatenate([xyz, colors, labels], axis=1))

In [6]:
xyzRgbLabel = np.load('data/004.npy')
print(xyzRgbLabel.shape)

xyzrgb = xyzRgbLabel[:, :6]
xyz = xyzRgbLabel[:, :3]
rgb = xyzRgbLabel[:, 3:6]
label = xyzRgbLabel[:, 6]
# print(xyzrgb)

(202742, 7)


In [8]:
from util.vis.helper_tool import Plot

Plot.draw_pc_sem_ins(xyz, label, None, 1)
# Plot.draw_pc(xyzrgb)

TypeError: unsupported operand type(s) for /: 'list' and 'int'